In [6]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.amp import GradScaler, autocast
from torch.nn.utils import clip_grad_norm_
import mmap
import random
import pickle
import argparse
import time


parser = argparse.ArgumentParser(description='This is a demonstration program')

# Here we add an argument to the parser, specifying the expected type, a help message, etc.
# parser.add_argument('-batch_size', type=str, required=True, help='Please provide a batch_size')

# args = parser.parse_args()

# Now we can use the argument value in our program.
# print(f'batch size: {args.batch_size}')
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# batch_size = args.batch_size # to use the batch_size cmd arg -> python file_name.py -batch_size 32
batch_size = 128
block_size = 128
max_iters = 400
learning_rate = 8e-4
eval_iters = 100
#eval_sample = 25
n_embd = 384
n_head = 8
n_layer = 8
dropout = 0.15

print(device)

cuda


In [7]:
chars = ""
with open("cleanVocab.txt", 'r', encoding='utf-8') as f:
        text = f.read()
        chars = sorted(list(set(text)))
        
vocab_size = len(chars)
print(chars)
print(vocab_size)

['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~', '£', '\xad', '½', 'ß', 'ø', 'α', 'β', 'δ', 'λ', 'μ', 'π', 'ρ', 'σ', 'τ', 'а', 'е', 'з', 'о', 'р', 'с', '–', '—', '‘', '’', '“', '”', '•', '…', '€', '−', '�', '🙂']
127


In [8]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

In [9]:
# memory map for using small snippets of text from a single file of any size
def get_random_chunk(split):
    filename = "output_train_pruned.txt" if split == 'train' else "output_val_pruned.txt"
    with open(filename, 'rb') as f:
        with mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ) as mm:
            # Determine the file size and a random position to start reading
            file_size = len(mm)
            start_pos = random.randint(0, (file_size) - block_size*batch_size)

            # Seek to the random position and read the block of text
            mm.seek(start_pos)
            block = mm.read(block_size*batch_size-1)

            # Decode the block to a string, ignoring any invalid byte sequences
            decoded_block = block.decode('utf-8', errors='ignore').replace('\r', '')
            
            # Train and test splits
            data = torch.tensor(encode(decoded_block), dtype=torch.long)
            
    return data


def get_batch(split):
    data = get_random_chunk(split)
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [10]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

# [1, 0, 0]
# [1, 0.6, 0]
# [1, 0.6, 0.4]
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1) # (B, T, F) -> (B, T, [h1, h1, h1, h1, h2, h2, h2, h2, h3, h3, h3, h3])
        out = self.dropout(self.proj(out))
        return out
    

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)
    
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x + y)
        return x
    
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
        
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, index, targets=None):
        B, T = index.shape
        
        
        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(index) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            index_cond = index[:, -block_size:]
            # get the predictions
            logits, loss = self.forward(index_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = GPTLanguageModel(vocab_size)

print('loading model parameters...')
with open('model-completed-06.pkl', 'rb') as f:
     model = pickle.load(f)
print('loaded successfully!')

m = model.to(device)

loading model parameters...


EOFError: Ran out of input

In [14]:
@torch.no_grad()
def estimate_loss():
    model.eval()
    device = next(model.parameters()).device
    out = {}
    
    for split in ['train', 'val']:
        losses = []
        for _ in range(eval_iters):
            X, Y = get_batch(split)
            X, Y = X.to(device), Y.to(device)
            with autocast(device_type='cuda', dtype=torch.float16):
                logits, loss = model(X, Y)
            losses.append(loss.item())
        out[split] = torch.tensor(losses).mean().item()
    
    model.train()
    return out

In [ ]:
torch.backends.cudnn.benchmark = True

In [13]:
# Create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
scaler = GradScaler()

def save_model(model, filename='model-clean-08.pkl'):
    torch.save(model.state_dict(), filename)
    print(f'Model saved as {filename}')

try:
    start_time = time.time()
    print("Training started")
    for iter in range(max_iters):
        if iter % eval_iters == 0:
            losses = estimate_loss()
            print(f"step: {iter}, time: {(time.time() - start_time):.3f}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")
        
        # Sample a batch of data
        xb, yb = get_batch('train')
        
        # Evaluate the loss
        with autocast(device_type='cuda', dtype=torch.float16):
            logits, loss = model(xb, yb)
        
        # Backward pass and optimization
        optimizer.zero_grad(set_to_none=True)
        scaler.scale(loss).backward()
        
        # Gradient clipping
        scaler.unscale_(optimizer)
        clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        scaler.step(optimizer)
        scaler.update()
        
        time.sleep(0.001)
        
except KeyboardInterrupt:
    print("\nTraining interrupted. Saving model...")
    save_model(model)
    print("You can now safely stop the notebook.")
else:
    print(f"Final loss: {loss.item()}")
    save_model(model, 'model-completed-07.pkl')
    print('Training completed and model saved')

Training started
step: 0, time: 118.568, train loss: 1.371, val loss: 1.373
step: 200, time: 542.870, train loss: 1.370, val loss: 1.377
step: 400, time: 968.235, train loss: 1.373, val loss: 1.373
step: 600, time: 1393.837, train loss: 1.392, val loss: 1.399
step: 800, time: 1819.396, train loss: 1.366, val loss: 1.355
step: 1000, time: 2244.907, train loss: 1.366, val loss: 1.377
step: 1200, time: 2670.278, train loss: 1.378, val loss: 1.377
step: 1400, time: 3095.219, train loss: 1.384, val loss: 1.371
step: 1600, time: 3519.652, train loss: 1.363, val loss: 1.372
step: 1800, time: 3943.743, train loss: 1.378, val loss: 1.355
step: 2000, time: 4367.966, train loss: 1.348, val loss: 1.368
step: 2200, time: 4792.342, train loss: 1.361, val loss: 1.351
step: 2400, time: 5216.507, train loss: 1.368, val loss: 1.366
step: 2600, time: 5640.767, train loss: 1.366, val loss: 1.345
step: 2800, time: 6065.963, train loss: 1.350, val loss: 1.348
step: 3000, time: 6491.648, train loss: 1.353, v

In [11]:
prompt = 'Once Upon a time,'
context = torch.tensor(encode(prompt), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context.unsqueeze(0), max_new_tokens=100)[0].tolist())
print(generated_chars)

NameError: name 'm' is not defined

In [22]:
print('loading model parameters...')
with open('model-clean-04.pkl', 'rb') as f:
     model = pickle.load(f)
print('loaded successfully!')

m = model.to(device)

loading model parameters...
loaded successfully!


In [23]:
start_time = time.time()
print("evaluating losses...")
losses = estimate_loss()

print(f"time: {(time.time() - start_time):.3f}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

evaluating losses...
time: 59.830, train loss: 1.459, val loss: 1.457


In [16]:
#with open('model-interrupted.pkl', 'wb') as f:
#    pickle.dump(model, f)
#print('Model saved')

In [29]:
class TensorLoader:
    def __init__(self):
        self.tensors = {}

    def persistent_id(self, obj):
        if isinstance(obj, torch.Tensor):
            key = len(self.tensors)
            self.tensors[key] = obj
            return ("torch.Tensor", key)
        return None

    def persistent_load(self, pid):
        type_tag, key = pid
        if type_tag == "torch.Tensor":
            return self.tensors[key]
        return None

# Create an instance of the TensorLoader
tensor_loader = TensorLoader()

# Use this for loading
with open('model-completed-05.pkl', 'rb') as f:
    unpickler = pickle.Unpickler(f)
    unpickler.persistent_load = tensor_loader.persistent_load
    model = unpickler.load()